In [ ]:
import csv
import random

# Base phrases that are gonna be used for data generation
base_phrases = {
    'book_flight': [
        "book a flight", "I need to book a flight", "reserve a flight ticket", "I want to fly",
        "schedule a flight for me", "can I get a flight", "arrange a flight", "need a flight booked"
    ],
    'cancel_flight': [
        "cancel my flight", "I need to cancel my booking", "please cancel my flight reservation",
        "abort my flight reservation", "I have to cancel my flight", "stop my flight booking"
    ],
    'query_flight': [
        "show me available flights", "I want to see flight options", "what flights are available",
        "need information on flights", "flight availability", "find flights for me"
    ]
}

# Generating variations
def generate_variations(phrases, num_variations=20):
    variations = []
    for phrase in phrases:
        for _ in range(num_variations):
            # All different kinds of data augmentation
            variation = phrase
            if random.choice([True, False]):
                variation = "please " + variation
            if random.choice([True, False]):
                variation = variation + " now"
            if random.choice([True, False]):
                variation = "Can you " + variation
            if random.choice([True, False]):
                variation = variation.replace("I", "Could I")
            variations.append(variation)
    return variations

# Generating dataset
dataset = []
for category, phrases in base_phrases.items():
    variations = generate_variations(phrases)
    for v in variations:
        dataset.append((v, category))

random.shuffle(dataset)

# Setting amount of entrys
while len(dataset) < 1000:
    for category, phrases in base_phrases.items():
        variations = generate_variations(phrases, 1)
        for v in variations:
            dataset.append((v, category))
            if len(dataset) >= 1000:
                break
    if len(dataset) >= 1000:
        break

# Saving into csv
filename = 'flight_booking_dataset.csv'
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Phrase', 'Category'])
    for entry in dataset:
        writer.writerow(entry)

print(f"Dataset saved as {filename}. Total entries: {len(dataset)}")


Dataset saved as flight_booking_dataset.csv. Total entries: 1000


In [ ]:
!pip install tensorflow transformers


In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.utils import to_categorical

# Loading dataset
df = pd.read_csv('flight_booking_dataset.csv')

# Loading tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Encodeing labels
label_dict = {'book_flight': 0, 'cancel_flight': 1, 'query_flight': 2}
df['label'] = df['Category'].replace(label_dict)

# Spliting dataset
X_train, X_test, y_train, y_test = train_test_split(df['Phrase'], df['label'], test_size=0.2, random_state=42)

# Tokenizeing and encoding sentences
def encode_sentences(tokenizer, sentences, max_length=128):
    input_ids, attention_masks = [], []

    for sentence in sentences:
        encoded_dict = tokenizer.encode_plus(
            sentence,
            add_special_tokens = True,
            max_length = max_length,
            pad_to_max_length = True,
            return_attention_mask = True,
            return_tensors = 'tf',
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = tf.concat(input_ids, 0)
    attention_masks = tf.concat(attention_masks, 0)

    return input_ids, attention_masks

max_length = 128 # This length should be sufficient for the task
train_input_ids, train_attention_masks = encode_sentences(tokenizer, X_train, max_length)
test_input_ids, test_attention_masks = encode_sentences(tokenizer, X_test, max_length)

# Converting labels to categorical
train_labels = to_categorical(y_train, num_classes=3)
test_labels = to_categorical(y_test, num_classes=3)
# Loading pre-trained BERT model. A bit of an overkill.
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Compiling the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5, epsilon=1e-08)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Training the model. We dont need to train it a lot, since bert is very capable on its own.
batch_size = 32
epochs = 3

model.fit(
    [train_input_ids, train_attention_masks], train_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=([test_input_ids, test_attention_masks], test_labels)
)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weight

Epoch 1/3
25/25 [==============================] - 81s 950ms/step - loss: 0.7275 - accuracy: 0.7700 - val_loss: 0.3329 - val_accuracy: 1.0000
Epoch 2/3
25/25 [==============================] - 21s 843ms/step - loss: 0.2204 - accuracy: 1.0000 - val_loss: 0.1020 - val_accuracy: 1.0000
Epoch 3/3
25/25 [==============================] - 23s 916ms/step - loss: 0.0700 - accuracy: 1.0000 - val_loss: 0.0345 - val_accuracy: 1.0000


In [ ]:
# Following are mock api functions
def book_flight():
    return "Flight booked successfully."

def cancel_flight():
    return "Flight cancelled successfully."

def query_flights():
    return "Here are the available flights."

# There we processing user input
def classify_intent(text):
    # Tokenizing the user input text
    input_ids, attention_mask = encode_sentences(tokenizer, [text])

    # Making a prediction
    predictions = model.predict([input_ids, attention_mask])

    # Getting the probabilities
    probabilities = tf.nn.softmax(predictions.logits, axis=-1)

    predicted_class = tf.argmax(probabilities, axis=1).numpy()[0]
    confidence = tf.reduce_max(probabilities, axis=1).numpy()[0]

    return predicted_class, confidence

# this function responsible for user talking to model loop. Making it a loop allows user to talk with model continuously
def interact_with_model():
    CONFIDENCE_THRESHOLD = 0.85  # Controls how sure model must be to ask for clarification

    while True:
        user_input = input("How can I assist you with your flight today? (Type 'exit' to quit) ")
        if user_input.lower() == 'exit':
            break

        # Classifying the intent of the user input and getting the confidence level
        intent, confidence = classify_intent(user_input)

        # Checking if the confidence is below the threshold
        if confidence < CONFIDENCE_THRESHOLD:
            print("I'm not quite sure what you are asking. Could you provide more details?")
            continue

        # Calling the mock API function
        if intent == 0:
            print(book_flight())
        elif intent == 1:
            print(cancel_flight())
        elif intent == 2:
            print(query_flights())
        else:
            print("I'm not sure what you're asking. Could you please clarify?")

# Run the interaction function
interact_with_model()


How can I assist you with your flight today? (Type 'exit' to quit) what are flight from paris ?
1/1 [==============================] - 0s 74ms/step
I'm not quite sure what you are asking. Could you provide more details?
How can I assist you with your flight today? (Type 'exit' to quit) show mw flight frop paris 
1/1 [==============================] - 0s 54ms/step
Here are the available flights.
How can I assist you with your flight today? (Type 'exit' to quit) exit
